In [1]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [2]:
model = 'gpt-4o-mini'
db_name = 'vector_db'

In [3]:
load_dotenv()
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
folders = glob.glob('knowledge-base/*')
text_loader_kwargs = {'encoding': 'utf-8'}

documents = []

for folder in folders :
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata['doc_type'] = doc_type
        documents.append(doc)

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [6]:
len(chunks)

123

In [7]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: company, contracts, employees, products


In [8]:
embeddings = OpenAIEmbeddings()

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f'Vector Store created {vectorstore._collection.count()} documents')

Vector Store created 123 documents


In [9]:
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


In [ ]:
llm = ChatOpenAI(temperature=0.7, model_name=model)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vectorstore.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

C:\Users\Marwa\AppData\Local\Temp\ipykernel_24396\546388679.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [11]:
query = 'Can you descripe Insurelm in few sentences'
result = converstaion_chain.invoke({'question':query})
print(result['answer'])

Insurellm is an innovative insurance tech firm founded in 2015, known for its four software products: Carllm, Homellm, Rellm, and Marketllm. With a workforce of 200 employees and 12 offices across the US, Insurellm has expanded rapidly and serves over 300 clients worldwide. The company focuses on providing cutting-edge solutions for the insurance and reinsurance sectors, emphasizing customer support, training, and continuous improvement through updates and feedback.


In [12]:
def chat(message,history):
    result = converstaion_chain.invoke({'question':message})
    return result['answer']

In [13]:
view = gr.ChatInterface(fn=chat).launch()

d:\LLM-Engineering\llm-engineering\Lib\site-packages\gradio\chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [18]:
from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(temperature=0.7, model_name=model)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vectorstore.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, memory= memory, retriever= retriever, callbacks=[StdOutCallbackHandler()])

query = 'who recieved IIOTY award in 2023?'
result = conversation_chain.invoke({'question':query})
answer = result['answer']
print('Answer: ',answer)



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
## Annual Performance History
- **2020:**  
  - Completed onboarding successfully.  
  - Met expectations in delivering project milestones.  
  - Received positive feedback from the team leads.

- **2021:**  
  - Achieved a 95% success rate in project delivery timelines.  
  - Awarded "Rising Star" at the annual company gala for outstanding contributions.  

- **2022:**  
  - Exceeded goals by optimizing existing backend code, improving system performance by 25%.  
  - Conducted training sessions for junior developers, fostering knowledge sharing.  

- **2023:**  
  - Led a major overhaul of the API internal architecture, enhancing security proto

In [ ]:
llm = ChatOpenAI(temperature=0.7, model_name=model)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 30})
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, memory=memory, retriever=retriever)  

query = 'who recieved IIOTY award in 2023?'
result = conversation_chain.invoke({'question':query})
answer = result['answer']
print('Answer: ',answer)

Answer:  Maxine received the IIOTY (Insurellm Innovator of the Year) award in 2023.
